# Starter Notebook

### You can find the full article about this notebook in **[Towards AI here](https://medium.com/towards-artificial-intelligence/n-dimensional-dicom-volumes-with-imageio-e494e2bb3abe)**. Feel free to leave any feedback or questions in the comments!


In [ ]:
import imageio.v2 as iio
import numpy as np
from matplotlib import pyplot as plt
import scipy.ndimage as ndi
from skimage import exposure

from ipywidgets import interact
import ipywidgets as widgets

We'll first load a single DICOM file as an image object using `imageio`. Once we've converted the DICOM file to an imageio image object, we can plot it with matplotlib.pyplot, just as we would any other image array.


In [ ]:
im = iio.imread('/kaggle/input/cranial-ct/Cranial CT/1.3.6.1.4.1.5962.99.1.2786334768.1849416866.1385765836848.149.0.dcm')

# Print image attributes
print('Image type:', type(im))
print('Shape of image array:', im.shape)

# Show image
plt.imshow(im, cmap= 'gray')

In [ ]:
# plot 3 versions of chest dcm with augmentations
plt.rcParams['figure.figsize'] = (14, 8)
fig, axes = plt.subplots(nrows= 1, ncols= 3)
axes[0].imshow(im, cmap= 'gray')
axes[1].imshow(im, vmin= -200, vmax= 200, cmap= 'gray')
axes[2].imshow(im, vmin= -200, vmax= 200, cmap= 'magma')
axes[0].set_title('Original')
axes[1].set_title('High Contrast')
axes[2].set_title('Colored')
for ax in axes:
  ax.axis('off')
plt.show()

## Reading DICOM metadata
Note that `imageio` aggregates metadata across volumes when appropriate. This you can either call the `meta` attribute on a **single image** or a **full volume**.

In [ ]:
# print metadata dictionary
im.meta

In [ ]:
# print metadata keys
print(im.meta.keys())

In [ ]:
# print a few metadata values
print(im.meta.PatientSex)
print(im.meta.StudyDate)

## Reading a DICOM volume with `np.stack`

In [ ]:
# read in each image individually
im1 = iio.imread('/kaggle/input/cranial-ct/Cranial CT/1.3.6.1.4.1.5962.99.1.2786334768.1849416866.1385765836848.177.0.dcm')
im2 = iio.imread('/kaggle/input/cranial-ct/Cranial CT/1.3.6.1.4.1.5962.99.1.2786334768.1849416866.1385765836848.178.0.dcm')
im3 = iio.imread('/kaggle/input/cranial-ct/Cranial CT/1.3.6.1.4.1.5962.99.1.2786334768.1849416866.1385765836848.179.0.dcm')

# stack the images
vol = np.stack([im1, im2, im3])

# confirm the shape of the new array
vol.shape

## Reading a DICOM volume with `imageio.volread`
We pass a single file path to **`imageio.imread`** to retrieve metadata, but we can pass the entire DICOM volume to **`imageio.volread`**

In [ ]:
# read DICOM volume with imageio's volread
cranial_vol = iio.volread('/kaggle/input/cranial-ct/Cranial CT/1.3.6.1.4.1.5962.99.1.2786334768.1849416866.1385765836848.149.0.dcm')

# print image volume and attributes
print('Volume metadata: {}'.format(cranial_vol.meta.keys()))
print('Volume shape: {}'.format(cranial_vol.shape))

In [ ]:
# plot all chest DICOM images in one view
fig, axes = plt.subplots(nrows=1, ncols=5)
[axes[x].imshow(cranial_vol[x], vmin= -300, vmax = 300) for x in range(5)]
[ax.axis('off') for ax in axes]
plt.show()

In [ ]:
# define sampling resolution along each axis
d0, d1, d2 = cranial_vol.meta.sampling
# define shape along each axis
n0, n1, n2 = cranial_vol.meta.shape

# calculate field of view along each axis
axial_fov = n0 * d0
coronal_fov = n1 * d1
sagittal_fov = n2 * d2

print('Axial FOV: {} \n Coronal FOV: {} \n Sagittal FOV: {} \n'.format(axial_fov, 
                                                                       coronal_fov, 
                                                                       sagittal_fov))

In [ ]:
# calculate axial aspect ratio
axial_aspect = d1 / d2

# calculate sagittal aspect ratio
sagittal_aspect = d0 / d1

# calculate coronal aspect ratio
coronal_aspect = d0 / d2

In [ ]:
print('Axial aspect ratio: {} \n Coronal aspect ratio: {} \n Sagittal aspect ratio: {} \n'.format(axial_aspect, 
                                                                                                  coronal_aspect, 
                                                                                                  sagittal_aspect))

In [ ]:
@widgets.interact(brain=(0,225))
def brain_DICOM(brain = 0):
    fig, ax = plt.subplots(1,1, figsize = (8,8))
    ax.imshow(cranial_vol[brain,:,:], vmin= -100, vmax=100)
    ax.axis('off')

**Note** that in the following two plots (cranial and sagittal views), we invert the y axis to take the upside-down images and turn them right-side-up.

In [ ]:
# ipywidget wrapper
@widgets.interact(coronal_slice=(0,n1-1),
                  sagittal_slice=(0,n2-1))

# walkthrough function
def slicer(coronal_slice=100, sagittal_slice=100):
    fig, ax = plt.subplots(1, 2, figsize=(20, 15))

    # visualize coronal plane
    ax[0].imshow(cranial_vol[:,coronal_slice,:], vmin=-100, vmax=100,aspect= coronal_aspect)
    ax[0].axis('off')
    ax[0].set_title('Coronal')
    ax[0].invert_yaxis()

    # visualize sagital plane
    ax[1].imshow(cranial_vol[:,:,sagittal_slice], vmin=-100, vmax=100,aspect= sagittal_aspect)
    ax[1].axis('off')
    ax[1].set_title('Sagittal')
    ax[1].invert_yaxis()

Now let's compare the difference between Sagittal view with its original aspect ratio and with its aspect ratio corrected:

In [ ]:
# ipywidget wrapper
@widgets.interact(sagital_slice_pre=(0,225),
                  sagittal_slice=(0,225))

# walkthrough function
def slicer(sagittal_slice_pre=100, sagittal_slice=100):
    fig, ax = plt.subplots(1, 2, figsize=(20, 15))

    # visualize coronal plane
    ax[0].imshow(cranial_vol[:,:,sagittal_slice_pre], vmin=-100, vmax=100)
    ax[0].axis('off')
    ax[0].set_title('Sagittal plane with original axial aspect ratio')
    ax[0].invert_yaxis()

    # visualize sagital plane
    ax[1].imshow(cranial_vol[:,:,sagittal_slice], vmin=-100, vmax=100,aspect= sagittal_aspect)
    ax[1].axis('off')
    ax[1].set_title('Sagittal plane with sagittal aspect ratio')
    ax[1].invert_yaxis()

### To learn more, please visit **[the full article here](https://medium.com/towards-artificial-intelligence/n-dimensional-dicom-volumes-with-imageio-e494e2bb3abe).**